In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class DeepfakeDetectorCNN(nn.Module):
    def __init__(self):
        super(DeepfakeDetectorCNN, self).__init__()
        
        # Layer 1 :256x256 -> 128x128 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Layer 2 : 64x64
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Layer 3 : 32x32
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Layer 4 :  16x16
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Layer 5 :  8x8
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully Connected layers
        # 512 filtres * 8 * 8 pixels restants
        self.fc1 = nn.Linear(512 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 1) # Sortie binaire (Fake ou Real)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        
        x = x.view(-1, 512 * 8 * 8) # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available: ", torch.cuda.is_available())
print("GPU name: ", torch.cuda.get_device_name(0))
print(f"Training on device: {device}")

CUDA available:  True
GPU name:  NVIDIA GeForce RTX 4050 Laptop GPU
Training on device: cuda


In [11]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau



model = DeepfakeDetectorCNN().to(device)

criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-4) 


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

In [12]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

train_dataset = datasets.ImageFolder("../images/train", transform=train_transform)
val_dataset = datasets.ImageFolder("../images/val", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
print("Number of training samples: ", len(train_dataset))

Number of training samples:  47991


In [14]:
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# Parameters
num_epochs = 20
best_f1 = 0.0

print(f"Starting training on {device}...")

for epoch in range(num_epochs):
    # ================= TRAIN PHASE =================
    model.train()
    correct, total, running_loss = 0, 0, 0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

    for images, labels in train_bar:
        images = images.to(device)
        # Reshape labels to (batch_size, 1) and convert to float for BCELoss
        labels = labels.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Predictions: if output > 0.5 then 1 (Real), else 0 (Fake)
        preds = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        train_bar.set_postfix({
            "Loss": f"{loss.item():.4f}", 
            "Acc": f"{100*correct/total:.2f}%"
        })

    train_acc = 100 * correct / total

    # ================= VALIDATION PHASE =================
    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        for images, labels in val_bar:
            images = images.to(device)
            labels_float = labels.to(device).float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels_float)
            val_loss += loss.item()
            
            # Binary classification threshold
            preds = (outputs > 0.5).float()
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy()) # Keep original labels for metrics

    # Calculate Metrics
    # Pos_label=0 because we focus on detecting "Fake"
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, np.array(all_preds).flatten(), average='binary', pos_label=0
    )
    
    val_acc = 100 * np.sum(np.array(all_preds).flatten() == np.array(all_labels)) / len(all_labels)
    avg_val_loss = val_loss / len(val_loader)

    # Adaptive Learning Rate Update
    scheduler.step(avg_val_loss)

    print(f"\n--- Epoch {epoch+1} Summary ---")
    print(f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}% | Val Loss: {avg_val_loss:.4f}")
    print(f"Metrics (Fake Class 0) -> Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

    # ================= CHECKPOINT =================
    if f1 > best_f1: 
        best_f1 = f1
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "f1_score": f1,
            "val_loss": avg_val_loss
        }, "best_deepfake_detector.pt")
        print(f"⭐ New F1-Score record: {f1:.4f}! Model saved.\n")

Starting training on cuda...


Epoch 1/20 [Val]: 100%|██████████| 188/188 [00:08<00:00, 22.76it/s]



--- Epoch 1 Summary ---
Train Acc: 74.70% | Val Acc: 78.86% | Val Loss: 0.4541
Metrics (Fake Class 0) -> Precision: 0.7645 | Recall: 0.8343 | F1: 0.7979
⭐ New F1-Score record: 0.7979! Model saved.



Epoch 2/20 [Val]: 100%|██████████| 188/188 [00:08<00:00, 22.50it/s]



--- Epoch 2 Summary ---
Train Acc: 78.05% | Val Acc: 80.43% | Val Loss: 0.4274
Metrics (Fake Class 0) -> Precision: 0.7827 | Recall: 0.8427 | F1: 0.8116
⭐ New F1-Score record: 0.8116! Model saved.



...


--- Epoch 18 Summary ---
Train Acc: 88.74% | Val Acc: 88.91% | Val Loss: 0.2628
Metrics (Fake Class 0) -> Precision: 0.8852 | Recall: 0.8943 | F1: 0.8897
⭐ New F1-Score record: 0.8897! Model saved.



Epoch 19/20 [Val]: 100%|██████████| 188/188 [00:08<00:00, 22.15it/s]



--- Epoch 19 Summary ---
Train Acc: 89.02% | Val Acc: 88.91% | Val Loss: 0.2645
Metrics (Fake Class 0) -> Precision: 0.9084 | Recall: 0.8657 | F1: 0.8865


Epoch 20/20 [Val]: 100%|██████████| 188/188 [00:08<00:00, 21.53it/s]


--- Epoch 20 Summary ---
Train Acc: 89.18% | Val Acc: 88.85% | Val Loss: 0.2601
Metrics (Fake Class 0) -> Precision: 0.9004 | Recall: 0.8737 | F1: 0.8868
